In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.nn.functional import softmax
import torch
import json

In [2]:
small_ranking_model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-MiniLM-L-6-v2')
small_ranking_tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-MiniLM-L-6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
large_ranking_model = AutoModelForSequenceClassification.from_pretrained('cross-encoder/ms-marco-TinyBERT-L-6')
large_ranking_tokenizer = AutoTokenizer.from_pretrained('cross-encoder/ms-marco-TinyBERT-L-6')

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
def rerank(query, candidates, tokenizer, model):
    inputs = tokenizer([query] * len(candidates), candidates, padding=True, truncation=True, return_tensors="pt")
    outputs = model(**inputs)
    scores = softmax(outputs.logits, dim=-1)[:, 1]  # Use the relevance score
    reranked_results = torch.argsort(scores, descending=True)
    return reranked_results, scores

In [14]:
def load_json(file_name):
    with open(file_name, 'r') as f:
        return json.load(f)

In [22]:
corpus = load_json("/content/drive/MyDrive/corpus.json")
queries = load_json("/content/drive/MyDrive/queries.json")
qrels = load_json("/content/drive/MyDrive/qrels.json")
corpus_ids = list(corpus.keys())

In [17]:
def load_top_results(file_name):
    with open(file_name, 'r') as f:
        return json.load(f)

In [18]:
top_k_results = load_top_results("/content/drive/MyDrive/top_k_results.json")

In [19]:
query_id = list(top_k_results.keys())[0]
query_text = top_k_results[query_id]['query']
top_k_docs = top_k_results[query_id]['top_k_docs']
top_k_scores = top_k_results[query_id]['scores']

In [30]:
query = top_k_results[query_id]['query']

In [20]:
print(f"Query: {query_text}")
print(f"Top-10 Docs: {top_k_docs}")
print(f"Scores: {top_k_scores}")

Query: Am I required to have a lawyer create / oversee creation of my will?
Top-10 Docs: ['437100', '547388', '438080', '295142', '292324', '575381', '487343', '422994', '589862', '531665']
Scores: [0.672465443611145, 0.6704652309417725, 0.608589231967926, 0.579573392868042, 0.5727294087409973, 0.4928045868873596, 0.4843670129776001, 0.4665147066116333, 0.4618057608604431, 0.45188039541244507]


In [24]:
candidate_texts = [corpus[doc_id]['text'] for doc_id in top_k_docs]

In [31]:
def rerank(query, candidates, tokenizer, model):
    inputs = tokenizer([query] * len(candidates), candidates, padding=True, truncation=True, return_tensors="pt")

    outputs = model(**inputs)

    print(f"Logits shape: {outputs.logits.shape}")  # Add this line to see the shape

    scores = softmax(outputs.logits, dim=-1)

    if scores.shape[1] == 1:  # Single output
        scores = scores[:, 0]  # Get the single score (relevance score)
    else:
        scores = scores[:, 1]  # Get the positive class score

    reranked_results = torch.argsort(scores, descending=True)
    return reranked_results, scores


In [33]:
reranked_results, scores = rerank(query, candidate_texts, small_ranking_tokenizer, small_ranking_model)

Logits shape: torch.Size([10, 1])


In [34]:
print(f"\nReranked results for query '{query}':")
for idx in reranked_results:
    print(f"Reranked Document: {candidate_texts[idx.item()]} (Score: {scores[idx.item()].item()})")


Reranked results for query 'Am I required to have a lawyer create / oversee creation of my will?':
Reranked Document: "This is not intended as legal advice, and only covers general knowledge I have on the subject of wills as a result of handling my own finances.   Each state of the USA has its own laws on wills and trusts.  You can find these online.  For example, in Kentucky I found state laws here:  http://www.lrc.ky.gov/krs/titles.htm and Title XXXIV is about wills and trusts.  I would recommend reading this, and then talking to a lawyer if it is not crystal clear.   Generally, if a lawyer does not draft your will, then either (1) you have no will, or (2) you use a form or computer program to make a will, that must then be properly witnessed before it is valid.  If you don't have it witnessed properly, then you have no will.  In some states you can have a holographic will, which means a will in your own handwriting.  That's when you have that 3am heart attack, and you get out a pad